In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


keydf = pd.read_csv('Campaign-keywords-0221205.csv')

keydf.drop(columns='Unnamed: 0', inplace=True)
keydf.replace('unknown', np.nan, inplace=True)
keydf.bid_strength.fillna('unknown', inplace=True)

In [2]:
keydf.head()

,keyword,source,bid_type,bid_strength,current_cpc_bid,suggested_bid,spend,attributed_sales,attributed_quantities,roas,impressions,clicks,ctr,average_cpc,ntb_attributed_sales,percent_ntb_attributed_sales
0,almond cheese,automated,default,green,1.24,1.24,0.2,0.0,0.0,0.0,2,1,50.000000,0.2,0.0,0.0
1,artichokes fresh,automated,default,unknown,1.24,NaN,0.2,0.0,0.0,0.0,58,1,1.724138,0.2,0.0,0.0
2,baba ganouj,manual,override,unknown,2.55,NaN,0.0,0.0,0.0,0.0,5,0,0.000000,0.0,0.0,0.0
3,babaganoush,manual,override,unknown,2.55,NaN,0.0,0.0,0.0,0.0,3,0,0.000000,0.0,0.0,0.0
4,baba ganoush,manual,override,unknown,2.55,NaN,0.0,0.0,0.0,0.0,50,0,0.000000,0.0,0.0,0.0


In [3]:
total_sales = keydf.attributed_sales.sum()
print('Total campaign sales: '+ str(total_sales))

num_keywords = keydf.shape[0]
print('Total number of keywords: ' + str(num_keywords))

Total campaign sales: 4325.9183297
Total number of keywords: 389


In [4]:
keydf.describe()

,current_cpc_bid,suggested_bid,spend,attributed_sales,attributed_quantities,roas,impressions,clicks,ctr,average_cpc,ntb_attributed_sales,percent_ntb_attributed_sales
count,389.000000,234.000000,389.000000,389.000000,389.000000,389.000000,389.000000,389.000000,389.000000,389.000000,389.000000,389.000000
mean,1.592674,2.011111,6.697738,11.120613,1.014567,3.100040,628.714653,4.573265,16.515647,0.995177,8.546696,23.596820
std,0.681321,1.252638,31.150544,43.025385,3.899429,13.024575,3369.445448,17.217167,29.658716,0.629913,34.602382,40.555688
min,0.420000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.220000,1.242500,0.690000,0.000000,0.000000,0.000000,6.000000,1.000000,0.630499,0.580000,0.000000,0.000000
50%,1.220000,1.750000,1.200000,0.000000,0.000000,0.000000,40.000000,1.000000,2.512563,1.060000,0.000000,0.000000
75%,1.890000,2.300000,2.270000,10.950000,1.000000,1.257176,186.000000,2.000000,14.285714,1.270000,8.990000,49.999998
max,4.900000,8.250000,472.240000,391.180001,35.333333,188.849985,49182.000000,216.000000,100.000000,3.247164,336.554993,100.000001


In [5]:
keydf.keyword.value_counts().reset_index().sort_values('keyword', ascending=False).head()

,index,keyword
0,almond cheese,2
23,vegan cream cheese,2
25,vegan dips,2
26,vegan spread,2
27,veggie dip,2


##### A total of \\$1623.0 dollars can be saved by removing keywords with ROAS < 1. This is 62% of the amount spent on ads for keywords with ROAS < 1, meaning that for every keywords with ROAS below 1, \\$1.0 dollars is invested and getting less than \\$1.0 dollars in return

In [6]:
# Total spent on adds
keydf.spend.sum()

2605.42

In [7]:
# Total ad spent ROAS < 1
keydf.spend[keydf.roas < 1].sum()

1623.0800000000002

In [8]:
# Total ad spent ROAS >= 1
keydf.spend[keydf.roas >= 1].sum()

982.34

In [9]:
# Total amount that can be saved by removing keywords with ROAS = 
keydf.spend[keydf.roas < 1].sum() - keydf.spend[keydf.roas >= 1].sum()

640.7400000000001

In [10]:
# % of ad spent in keywords with ROAS < 1
(1 - (keydf.spend.sum() - keydf.spend[keydf.roas < 1].sum()) / (keydf.spend.sum())) * 100

62.29629004152881

##### Most keywords with ROAS < 1 are automatically generated by Instacart

In [11]:
# keyword source distribution for keywords with ROAS < 1
auto_words_less_1 = keydf.source[keydf.roas < 1].value_counts()
auto_words_less_1

automated    212
manual        77
Name: source, dtype: int64

In [12]:
# key words source distribution for keywords with ROAS >=1
auto_words_more_1 = keydf.source[keydf.roas >= 1].value_counts()
auto_words_more_1

automated    65
manual       35
Name: source, dtype: int64

In [13]:
# % automated keywords with ROAS < 1
100 - ((auto_words_less_1.loc['manual'] / auto_words_less_1.loc['automated']) *100)

63.67924528301887

In [14]:
# % automated keywords with ROAS >= 1
100 - ((auto_words_more_1.loc['manual'] / auto_words_more_1.loc['automated']) *100)

46.15384615384615

##### Each keyword with a return on investment higher than the median (excluding those keywords with return on investment less than 1) is split into single words. The words are counted, therefore, a word that contains one or more of the best performing keywords could be a keyword a high potential return on investment (ROA).

In [15]:
# Selecting only whole campaign with ROAS > 0
keydf_pos = keydf.copy()
keydf_pos = keydf[keydf['roas'] >= 1]

In [16]:
keydf_median = keydf_pos[keydf_pos['roas'] > keydf_pos['roas'].median()]

keydf_pos_words = keydf_median.keyword.str.split().explode().value_counts().reset_index().rename({'index':"word","keyword":"count"},axis=1)
keydf_pos_words.head(20)
# keydf_pos_words.to_csv('words_count.csv')

,word,count
0,cheese,24
1,dip,12
2,free,7
3,vegan,6
4,dairy,5
5,cream,5
6,feta,4
7,cashew,4
8,dips,3
9,lactose,3


In [17]:
# Median ROA for keywords with ROA >=1
keydf_pos.roas.median()

5.421967638

In [18]:
# Total sales (revenue) 
keydf.attributed_sales[keydf.roas >= 1].sum()

3399.3333285

In [19]:
keydf.attributed_sales[keydf.roas < 1].sum()

926.5850012000001

In [20]:
keydf.attributed_sales[keydf.roas >= 1].sum() + keydf.attributed_sales[keydf.roas < 1].sum()

4325.9183297

### Recommendations

- Remove keywords with little return on investment (ROAS <1) because for every dollar paid in advertising, less than one dollar is made in profit.

- Experiment with keywords that have a combination of cheese, dip, cream, free, dairy, fat. For example, dairy free, fat free, lactose free.

- Most keywords with little return on invesment are automatically generated by Instacart (63%). These keywords are generated by considering a number of factors like what users are searching for. Therefore, it can be useful to find new keywords associated with the product. Some of the automated keywords are similar to manual generated keywords like cheese (manual), or cheeze (automated). The recommendation is to monitor the keywords weekly, and explore the possibility of adjusting the seetings when generating keywords by Instacart, if available.